In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [31]:
df = pd.read_csv('resultados_com_odds.csv')

In [1]:
from scipy.stats import norm

In [8]:
print(norm.cdf(-10.53947957/9))

0.12078867967701667


In [33]:
df = df.drop(columns=['event_id', 'odd_id','add_time','time_str'])

In [34]:
df

,date,time,home_team,away_team,score,handicap,home_od,away_od
0,2024-10-12,22:30:00,Sao Jose,Botafogo,70-81,-12.5,1.800,1.869
1,2024-10-12,22:30:00,Pinheiros,Pato Basquete,68-67,-1.5,1.833,1.833
2,2024-10-12,21:30:00,Mogi das Cruzes,Paulistano,72-67,6.5,1.869,1.800
3,2024-10-12,21:00:00,Minas,Brasilia,74-66,-13.5,1.714,2.000
4,2024-10-12,20:00:00,CR Vasco Da Gama,Flamengo,50-59,6.5,1.869,1.800
...,...,...,...,...,...,...,...,...
360,2025-06-03,23:00:00,Flamengo,Franca,95-98,-6.5,1.833,1.833
361,2025-06-07,20:00:00,Franca,Minas,65-69,-3.5,1.833,1.833
362,2025-06-12,22:30:00,Minas,Franca,78-94,-3.5,1.909,1.769
363,2025-06-14,20:00:00,Minas,Franca,73-83,-3.5,1.869,1.800


In [35]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [36]:
# parseia time em timedelta
df['time_delta'] = pd.to_timedelta(df['time'])

# cria datetime ajustado: date + time_delta menos 3 horas
df['date'] = df['date'] + df['time_delta'] - pd.Timedelta(hours=3)

# remove a coluna auxiliar
df.drop(columns=['time_delta', 'time'], inplace=True)


In [37]:
from scripts.constants.team_abrev import teams_abbrev
df['home_team'] = df['home_team'].map(teams_abbrev)
df['away_team'] = df['away_team'].map(teams_abbrev)

df[['Placar Mandante', 'Placar Visitante']] = df['score'].str.split('-', expand=True)
df[['Placar Mandante', 'Placar Visitante']] = df[['Placar Mandante', 'Placar Visitante']].astype(int)

df = df.drop(columns='score')

In [38]:
df = df.rename(columns={'home_team': 'Mandante', 'away_team': 'Visitante', 'handicap': 'Spread', 'home_od': 'Odd M', 'away_od': 'Odd V', 'date': 'Data'})


In [39]:
df['Resultado'] = np.where(df['Placar Mandante'] > df['Placar Visitante'], 1,0)
df['Total'] = df['Placar Mandante'] + df['Placar Visitante']
df['Saldo'] = df['Placar Mandante'] - df['Placar Visitante']
df['Hash'] = df['Mandante'].astype(str) + df['Data'].dt.strftime('%d%m%y')

In [41]:
df.to_csv('odds_24_25.csv', index=False)